In [861]:
!pip install tweepy --quiet

In [862]:
import sys
sys.executable

'c:\\Users\\14708\\anaconda3\\python.exe'

In [863]:
import numpy as np 
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt 
import seaborn as sns
import plotly.express as px 
%matplotlib inline

sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (10, 6)
matplotlib.rcParams['figure.facecolor'] = '#00000000'

pd.set_option('display.max_columns', 25)

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\14708\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\14708\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [864]:
main_df=pd.read_csv(r"C:\Users\14708\Downloads\Tata and Hyundai cars - data.xlsx - Data.csv")
df = main_df.copy()

## Data Exploration, Preparation

In [865]:
df.head()

,Date,Headline,URL,Opening Text,Hit Sentence,Source,Influencer,Country,Subregion,Language,Reach,Desktop Reach,Mobile Reach,Twitter Social Echo,Facebook Social Echo,Reddit Social Echo,National Viewership,Engagement,Sentiment,Keywords,State
0,28-Sep-2023 02:08PM,Kia और Hyundai ने वापस बुलाईं अपनी 35 लाख कारे...,https://hindi.pardaphash.com/kia-and-hyundai-r...,Kia and Hyundai Recalled Cars: वाहन निर्माता क...,Kia and Hyundai Recalled Cars: वाहन निर्माता क...,Pardaphash,NaN,India,Delhi,Hindi,31307.0,1279.0,30028.0,0.0,0.0,0.0,0,NaN,Neutral,"Hyundai,Cars",Delhi
1,28-Sep-2023 01:46PM,"Tata Punch EV spied, may compete with MG Comen...",https://economictimes.indiatimes.com/industry/...,Tata Motors is reportedly developing an electr...,... Punch EV is expected to compete with the M...,The Economic Times,NaN,India,National,English,15403059.0,6171732.0,9231327.0,0.0,0.0,0.0,0,NaN,Neutral,"Tata,cars",Maharashtra
2,28-Sep-2023 01:30PM,Tata Avinya Reviews On Road Price 2023 Mileage...,https://badisoch.in/automobile-news/tata-aviny...,... (adsbygoogle = window.adsbygoogle || []).p...,"... , despite most carmakers focusing on large...",Badi Soch,NaN,India,NaN,English,1128280.0,171549.0,956731.0,0.0,0.0,0.0,0,NaN,Neutral,"cars,Tata",NaN
3,28-Sep-2023 01:24PM,NaN,http://twitter.com/automobilindia8/statuses/17...,NaN,Engine Fire-Related Risks Force Kia & Hyundai ...,Twitter,@automobilindia8,India,NaN,English,2.0,NaN,NaN,NaN,NaN,NaN,0,0.0,Neutral,"Hyundai,Cars,hyundai",Karnataka
4,28-Sep-2023 01:22PM,NaN,http://twitter.com/MalakpetD/statuses/17073021...,NaN,"@srinualavilli @BMTC_BENGALURU Mahindra, Tata,...",Twitter,@MalakpetD,India,NaN,English,506.0,NaN,NaN,NaN,NaN,NaN,0,0.0,Neutral,"Tata,cars",Telangana


In [866]:
df.iloc[0]

Date                                                  28-Sep-2023 02:08PM
Headline                Kia और Hyundai ने वापस बुलाईं अपनी 35 लाख कारे...
URL                     https://hindi.pardaphash.com/kia-and-hyundai-r...
Opening Text            Kia and Hyundai Recalled Cars: वाहन निर्माता क...
Hit Sentence            Kia and Hyundai Recalled Cars: वाहन निर्माता क...
Source                                                         Pardaphash
Influencer                                                            NaN
Country                                                             India
Subregion                                                           Delhi
Language                                                            Hindi
Reach                                                             31307.0
Desktop Reach                                                      1279.0
Mobile Reach                                                      30028.0
Twitter Social Echo                   

In [867]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1528 entries, 0 to 1527
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Date                  1528 non-null   object 
 1   Headline              710 non-null    object 
 2   URL                   1528 non-null   object 
 3   Opening Text          710 non-null    object 
 4   Hit Sentence          1528 non-null   object 
 5   Source                1528 non-null   object 
 6   Influencer            1065 non-null   object 
 7   Country               1528 non-null   object 
 8   Subregion             321 non-null    object 
 9   Language              1528 non-null   object 
 10  Reach                 1519 non-null   float64
 11  Desktop Reach         710 non-null    float64
 12  Mobile Reach          710 non-null    float64
 13  Twitter Social Echo   710 non-null    float64
 14  Facebook Social Echo  710 non-null    float64
 15  Reddit Social Echo   

In [868]:
# Checking for Null values
df.isna().sum()

Date                       0
Headline                 818
URL                        0
Opening Text             818
Hit Sentence               0
Source                     0
Influencer               463
Country                    0
Subregion               1207
Language                   0
Reach                      9
Desktop Reach            818
Mobile Reach             818
Twitter Social Echo      818
Facebook Social Echo     818
Reddit Social Echo       818
National Viewership        0
Engagement              1176
Sentiment                  0
Keywords                   0
State                    500
dtype: int64

In [869]:
df = df[['Date', 'Headline', 'URL', 'Opening Text', 'Hit Sentence', 'Source',
       'Influencer', 'Country', 'Subregion', 'State', 'Language', 'Sentiment', 'Keywords',
       'Reach', 'Desktop Reach', 'Mobile Reach', 'Twitter Social Echo',
       'Facebook Social Echo', 'Reddit Social Echo', 'National Viewership',
       'Engagement'
       ]]

In [870]:
print('State Missing:',df['State'].isna().sum())
print('Subregion Missing:',df['Subregion'].isna().sum())

# These two are similar columns wioth large amounts of null values, thus one good way is to fill the null values with the values in the relevant column.
df['State'].fillna(df['Subregion'], inplace=True)
df['Subregion'].fillna(df['State'], inplace=True)

print(df['State'].isna().sum())
print(df['Subregion'].isna().sum())

State Missing: 500
Subregion Missing: 1207
497
497


In [871]:
df['State'].unique()

array(['Delhi', 'Maharashtra', nan, 'Karnataka', 'Telangana',
       'Madhya Pradesh', 'Uttar Pradesh', 'West Bengal', 'Chhattisgarh',
       'Kerala', 'Tamil Nadu', 'Rajasthan', 'Punjab', 'Odisha',
       'Chandigarh', 'Uttarakhand', 'Jammu and Kashmir', 'Gujarat',
       'Bihar', 'Haryana', 'Jharkhand', 'Andhra Pradesh',
       'Arunachal Pradesh', 'Assam', 'National', 'Tripura', 'Goa',
       'Mizoram', 'Himachal Pradesh', 'Puducherry'], dtype=object)

In [872]:
# Check for duplicates
df.duplicated().sum()

0

In [873]:
# categorical columns
cat_cols = df.select_dtypes('object').columns
print(cat_cols)

# numerical columns
numeric_cols = df.select_dtypes(include=np.number).columns
print(numeric_cols)

Index(['Date', 'Headline', 'URL', 'Opening Text', 'Hit Sentence', 'Source',
       'Influencer', 'Country', 'Subregion', 'State', 'Language', 'Sentiment',
       'Keywords'],
      dtype='object')
Index(['Reach', 'Desktop Reach', 'Mobile Reach', 'Twitter Social Echo',
       'Facebook Social Echo', 'Reddit Social Echo', 'National Viewership',
       'Engagement'],
      dtype='object')


In [874]:
df.isna().sum()

Date                       0
Headline                 818
URL                        0
Opening Text             818
Hit Sentence               0
Source                     0
Influencer               463
Country                    0
Subregion                497
State                    497
Language                   0
Sentiment                  0
Keywords                   0
Reach                      9
Desktop Reach            818
Mobile Reach             818
Twitter Social Echo      818
Facebook Social Echo     818
Reddit Social Echo       818
National Viewership        0
Engagement              1176
dtype: int64

In [875]:
df[df['Source']=='Twitter'].isna().sum()

Date                      0
Headline                818
URL                       0
Opening Text            818
Hit Sentence              0
Source                    0
Influencer                0
Country                   0
Subregion               129
State                   129
Language                  0
Sentiment                 0
Keywords                  0
Reach                     9
Desktop Reach           818
Mobile Reach            818
Twitter Social Echo     818
Facebook Social Echo    818
Reddit Social Echo      818
National Viewership       0
Engagement              504
dtype: int64

-----------------------------------------------------------------------------------------------------------------------------------------------------------

In [876]:
len(df['Keywords'].unique())

65

In [877]:
print(len(df['Keywords'].str.lower().unique()))
df['Keywords']=df['Keywords'].str.lower()
df['Keywords'].str.lower().unique()

28


array(['hyundai,cars', 'tata,cars', 'cars,tata', 'hyundai,cars,hyundai',
       'cars,hyundai', 'hyundai,cars,cars', 'cars,tata,hyundai',
       'cars,hyundai,tata', 'tata,cars,tata', 'cars,cars,tata',
       'cars,hyundai,cars', 'cars,hyundai,cars,tata', 'tata,cars,hyundai',
       'tata,cars,cars', 'cars,tata,tata,hyundai',
       'hyundai,cars,tata,cars', 'tata,tata,cars', 'tata,cars,tata,cars',
       'hyundai,cars,tata', 'hyundai,tata,cars', 'cars,cars,tata,tata',
       'cars,tata,tata', 'cars,tata,hyundai,cars', 'cars,tata,cars',
       'tata,cars,hyundai,cars', 'tata,hyundai,cars',
       'cars,hyundai,hyundai,cars', 'cars,tata,cars,hyundai'],
      dtype=object)

In [878]:
key_map = {
    'hyundai,cars':'hyundai', 'hyundai,cars,hyundai':'hyundai', 'cars,hyundai':'hyundai', 'hyundai,cars,cars':'hyundai',
    'tata,cars':'tata', 'cars,tata':'tata', 'cars,tata,hyundai':'hyundai_tata', 'cars,hyundai,tata':'hyundai_tata', 'tata,cars,tata':'tata',
    'cars,cars,tata':'tata', 'cars,hyundai,cars':'hyundai', 'cars,hyundai,cars,tata':'hyundai_tata', 'tata,cars,hyundai':'hyundai_tata',
    'tata,cars,cars':'tata', 'cars,tata,tata,hyundai':'hyundai_tata', 'hyundai,cars,tata,cars':'hyundai_tata', 'tata,tata,cars':'tata',
    'tata,cars,tata,cars':'tata','hyundai,cars,tata':'hyundai_tata', 'hyundai,tata,cars':'hyundai_tata', 'cars,cars,tata,tata':'tata',
    'cars,tata,tata':'tata', 'cars,tata,hyundai,cars':'hyundai_tata', 'cars,tata,cars':'tata','tata,cars,hyundai,cars':'hyundai_tata',
    'tata,hyundai,cars':'hyundai_tata', 'cars,hyundai,hyundai,cars':'hyundai', 'cars,tata,cars,hyundai':'hyundai_tata'
}

df['Keywords']=df['Keywords'].map(key_map)
df['Keywords'].unique()

array(['hyundai', 'tata', 'hyundai_tata'], dtype=object)

-----------------------------------------------------------------------------------------------------------------------------------------------------------

In [879]:
# Converting Hit Scentence to english for potential deep analysis in the future and to prevent data loss
df['Language'].value_counts()

English    1507
Hindi        14
Bengali       6
Marathi       1
Name: Language, dtype: int64

In [880]:
# Define a function to check if Hit Scentence column elements are not in English
def is_not_english(text):
    try:
        detected_language = detect(text)
        return detected_language != "en"
    except:
        return True  # If language detection fails, assume it's not English

# Apply the function to the 'Hit Sentence' column and create a boolean mask
non_english_mask = df['Hit Sentence'].apply(is_not_english)

# Filter the DataFrame to get rows with non-English text
non_english_texts = df[non_english_mask]

# Display the non-English texts
print(len(non_english_texts), ', ','This seems a small number, so lets translate these 26 elements into english')

26 ,  This seems a small number, so lets translate these 26 elements into english


In [881]:
def translate_to_english(text):
    translator = Translator(to_lang="en")
    translation = translator.translate(text)
    return translation

translations = []

for text in non_english_texts['Hit Sentence']:
    translation = translate_to_english(text)
    translations.append(translation)

non_english_texts['Translated Text'] = translations

translations

C:\Users\14708\AppData\Local\Temp\ipykernel_22564\2576393630.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_english_texts['Translated Text'] = translations


['Kia and Hyundai Recalled Cars: वाहन निर्माता किआ (Kia) और हुंडई (Hyundai) ने अमेरिका में अपनी लगभग 34 लाख कारों को वापस बुलाया है। कंपनिय ...',
 '"Hyundai Exter: नया एसयूवी आईएसयूवी मॉडल को लेकर भारतीय बाजार में हाई बुकिंग्स के साथ उतरा" https://t.co/7ms3tly1BC हमारे whatsapp ग्रुप से जुड़ने के लिए निचे दिए लिंक को टच करें https://t.co/OkFv3lB6d2 #hyundaiindia #hyundai #i #hyundaii #hyundaicreta #cars #creta https://t.co/WhBt9goPVG',
 'Top 5 Affordable Electric Cars In India हमारी इस लिस्ट में हाल ही लॉन्च हुई MG Comet EV से लेकर Tata Nexon EV तक शामिल है। एमजी कॉमेट ईवी भ ...',
 '... Hyundai Cars Discount: हुंडई मोटर इंडिया लिमिटेड (Hyundai Motor India Limited) अपनी बेहतरीन गाड़ियों के लिए देश में एक प्रचलित कार निर्मात ...',
 '... में सीधे टाटा पंच ईवी के साथ होगा. टाटा पंच ( Image Source : Tata Motors ) Upcoming Electric Cars: भारतीय ऑटोमोटिव उद्योग की तीन ऑट ...',
 'RT @gaadiwaadi: 3 All-New Hyundai Cars Likely Launching Next Year In India https://t.co/42gI42Kubj',
 '#Swift और #B

-----------------------------------------------------------------------------------------------------------------------------------------------------------

# Sentiment Analysis

In [882]:
df.groupby(['Keywords', 'Sentiment'], as_index=False).count()

,Keywords,Sentiment,Date,Headline,URL,Opening Text,Hit Sentence,Source,Influencer,Country,Subregion,State,Language,Reach,Desktop Reach,Mobile Reach,Twitter Social Echo,Facebook Social Echo,Reddit Social Echo,National Viewership,Engagement
0,hyundai,Negative,124,57,124,57,124,124,85,124,71,71,124,121,57,57,57,57,57,124,46
1,hyundai,Neutral,191,116,191,116,191,191,119,191,118,118,191,190,116,116,116,116,116,191,50
2,hyundai,Positive,162,133,162,133,162,162,72,162,85,85,162,162,133,133,133,133,133,162,29
3,hyundai_tata,Negative,6,2,6,2,6,6,5,6,4,4,6,5,2,2,2,2,2,6,2
4,hyundai_tata,Neutral,30,21,30,21,30,30,15,30,13,13,30,30,21,21,21,21,21,30,8
5,hyundai_tata,Positive,31,22,31,22,31,31,14,31,18,18,31,31,22,22,22,22,22,31,6
6,tata,Negative,128,8,128,8,128,128,124,128,105,105,128,127,8,8,8,8,8,128,66
7,tata,Neutral,629,192,629,192,629,629,501,629,482,482,629,626,192,192,192,192,192,629,89
8,tata,Positive,227,159,227,159,227,227,130,227,135,135,227,227,159,159,159,159,159,227,56


In [883]:
df['Hit Sentence'].tolist()

['Kia and Hyundai Recalled Cars: वाहन निर्माता किआ (Kia) और हुंडई (Hyundai) ने अमेरिका में अपनी लगभग 34 लाख कारों को वापस बुलाया है। कंपनिय ...',
 '... Punch EV is expected to compete with the MG Coment and Citroen C3. Tata Motors aims to increase its sales of green cars to 50% with an ...',
 '... , despite most carmakers focusing on large displays in their cars, Tata has opted for a small widescreen display integrated into the ...',
 'Engine Fire-Related Risks Force Kia & Hyundai to Recall Over 3 Million Cars in the US https://t.co/jI32U4ewC4 #Kia #hyundai #electriccars #Evcarsworld #evcars #enginefire #uscars #CarNews #automobile #automobilindia https://t.co/5KWFThTI7K',
 '@srinualavilli @BMTC_BENGALURU Mahindra, Tata, Reliance should come up flying cars for future generation with safety precautions.',
 '... on thе intеrnational stagе. Innovation and Sustainability: Thе Tata Group rеmains at thе forеfront of innovation and sustainability. ...',
 '... to consider the safest cars, or t

In [884]:
# Function for text preprocessing
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Tokenizing the text
    tokens = word_tokenize(text)
    
    # Removing punctuation and stop words
    tokens = [word for word in tokens if word not in string.punctuation]
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    
    # Joining the tokens back into a string
    processed_text = ' '.join(tokens)
    
    return processed_text


df['Hit Sentence'] = df['Hit Sentence'].apply(preprocess_text)

In [885]:
safety_keywords = [
    "Airbags",
    "Anti-lock Braking System", "ABS",
    "Electronic Stability Control","ESC",
    "Collision Avoidance Systems",
    "Adaptive Cruise Control",
    "Blind Spot Detection",
    "Lane Departure Warning",
    "Pedestrian Detection",
    "Rear-View Camera",
    "Crash Test Ratings",
    "Child Safety Features",
    "Advanced Driver Assistance Systems", "ADAS"
]

fuel_efficiency_keywords = [
    "Miles per Gallon", "MPG",
    "Hybrid Technology",
    "Eco Mode",
    "Start-Stop System",
    "Electric Vehicle Range",
    "Regenerative Braking",
    "Fuel Efficiency Standards",
    "Fuel Economy",
    "Green Technology",
    "Emissions",
    "Fuel-Efficient Engines",
    "Fuel Efficiency Comparisons"
]

accessories_keywords = [
    "Infotainment System",
    "Touchscreen Displays",
    "Apple CarPlay",
    "Android Auto",
    "Bluetooth Connectivity",
    "Premium Sound Systems",
    "Navigation",
    "Interior Comfort",
    "Leather Seats",
    "Sunroof",
    "Cargo Space",
    "Customization Options",
    "Connectivity Features"
]

ice_vs_electric_keywords = [
    "Internal Combustion Engine", "ICE",
    "Electric Vehicles", "EVs",
    "Battery Technology",
    "Charging Infrastructure",
    "Range Anxiety",
    "Carbon Emissions",
    "Plug-in Hybrid",
    "EV Incentives",
    "Charging Time",
    "Charging Stations",
    "Maintenance Costs",
    "Environmental Impact",
    "Resale Value"
]

safety_keywords = [word.lower() for word in safety_keywords]
fuel_efficiency_keywords = [word.lower() for word in fuel_efficiency_keywords]
accessories_keywords = [word.lower() for word in accessories_keywords]
ice_vs_electric_keywords = [word.lower() for word in ice_vs_electric_keywords]